In [ ]:
!pip install pymorphy3
!pip uninstall -y torch torchvision transformers
!pip install datasets torch torchvision transformers scikit-learn imbalanced-learn
!pip install evaluate rouge-score sacrebleu bert-score nltk

Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchvision 0.21.0
Uninstalling torchvision-0.21.0:
  Successfully uninstalled torchvision-0.21.0
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl (766.7 MB)
Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl (7.2 MB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)


In [ ]:
import evaluate
import gc
import nltk
import numpy as np
import os
import pandas as pd
import plotly.express as px
import pymorphy3
import re
import textwrap
import random
import torch
import torch.nn as nn
import transformers
import warnings
from bert_score import score as bert_score
from datasets import Dataset, concatenate_datasets, IterableDataset, load_dataset
from google.colab import drive
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from rouge_score import rouge_scorer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from transformers import (AutoModelForSequenceClassification,
                         AutoTokenizer,
                         DataCollatorForLanguageModeling,
                         DataCollatorForSeq2Seq,
                         get_cosine_schedule_with_warmup,
                         MBart50Tokenizer,
                         MBart50TokenizerFast,
                         MBartForConditionalGeneration,
                         TrainingArguments,
                         Trainer
)
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')
warnings.filterwarnings("ignore")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
input_dir = "extracted_text"
output_file = "cleaned_text.txt"
doc_tag_pattern = re.compile(r'<doc.*?>|</doc>', re.DOTALL)
with open(output_file, 'w', encoding='utf-8') as outfile:
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.startswith('wiki_'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as infile:
                    content = infile.read()
                    cleaned_content = doc_tag_pattern.sub('', content)
                    outfile.write(cleaned_content + '\n')


In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
input_file = "cleaned_text.txt"
output_file = "sentences.txt"
with open(output_file, 'w', encoding='utf-8') as outfile:
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            sentences = nltk.tokenize.sent_tokenize(line, language='russian')
            for sent in sentences:
                outfile.write(sent.strip() + '\n')

In [ ]:
import re

input_file = "/content/drive/MyDrive/Coursework/Bart_fine-tuning/sentences.txt"
output_file = "/content/drive/MyDrive/Coursework/Bart_fine-tuning/cleaned_russian_text.txt"

def clean_to_russian_only(text):
    cleaned_text = re.sub(r'[^А-Яа-яЁё\s]', '', text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        stripped = line.strip()
        if stripped:
            cleaned_line = clean_to_russian_only(stripped)
            if cleaned_line:
                outfile.write(cleaned_line + '\n')

In [ ]:
file_path = "/content/drive/MyDrive/Coursework/Bart_fine-tuning/cleaned_russian_text.txt"

num_lines = 0
file_size = 0
total_words = 0

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        num_lines += 1
        file_size += len(line)
        total_words += len(line.split())

avg_words_per_line = total_words / num_lines if num_lines > 0 else 0

print(f"Количество строк: {num_lines}")
print(f"Размер файла: {file_size} байт")
print(f"Среднее количество слов в строке: {avg_words_per_line:.2f}")


Количество строк: 155462964
Размер файла: 8383157790 байт
Среднее количество слов в строке: 6.81


In [ ]:
torch.cuda.empty_cache()
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="ru_RU")

max_length = 128

def gen_sentences(input_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            stripped = line.strip()
            if stripped:
                original_text = f"{tokenizer.bos_token}{stripped}{tokenizer.eos_token}"
                words = stripped.split()
                if len(words) > 5:
                    num_to_mask = max(1, int(len(words) * 0.2))
                    mask_indices = random.sample(range(len(words)), num_to_mask)
                    for idx in mask_indices:
                        words[idx] = "<mask>"
                    noisy_text = f"{tokenizer.bos_token}{' '.join(words)}{tokenizer.eos_token}"
                    yield {'input_text': noisy_text, 'target_text': original_text}

input_file = "/content/drive/MyDrive/Coursework/Bart_fine-tuning/cleaned_russian_text.txt"
dataset = IterableDataset.from_generator(lambda: gen_sentences(input_file))

def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")
    tokenizer.tgt_lang = "ru_RU"
    labels = tokenizer(text_target=examples["target_text"], max_length=128, truncation=True, padding="max_length", return_tensors="pt")["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["input_text", "target_text"])
model.to(torch.device("cuda"))

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    max_length=max_length
)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned",
    overwrite_output_dir=True,
    max_steps=50000,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=2,
    save_steps=2500,
    save_total_limit=2,
    logging_dir="/content/drive/MyDrive/Coursework/Bart_fine-tuning/logs",
    logging_steps=500,
    learning_rate=5e-5,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
trainer.train()
model.save_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: d-d-pushkarev-d-d (d-d-pushkarev-d-d-hse-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss
500,0.556500
1000,0.174600
1500,0.150200
2000,0.141700
2500,0.116200
3000,0.136400
3500,0.106700
4000,0.132300
4500,0.128500
5000,0.139900


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `pad

('/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned/sentencepiece.bpe.model',
 '/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned/added_tokens.json')

In [ ]:
dataset = load_dataset('d0rj/geo-reviews-dataset-2023', split='train')

README.md:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

(…)-00000-of-00001-49261e4e5a35a5a0.parquet:   0%|          | 0.00/171M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [ ]:
def preprocess_data(example):
    rating = example['rating']
    label = 1 if rating > 3 else 0
    return {'text': example['text'], 'label': label}

dataset = dataset.map(preprocess_data, remove_columns=['address', 'name_ru', 'rubrics', 'rating'])

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [ ]:
def oversample_dataset(dataset):
    positive = dataset.filter(lambda x: x['label'] == 1)
    negative = dataset.filter(lambda x: x['label'] == 0)
    num_pos= len(positive)
    num_neg = len(negative)
    oversample_factor = int(np.ceil(num_pos / num_neg))
    negative_oversampled = negative.select(np.tile(range(num_neg), oversample_factor)[:num_pos])
    balanced_dataset = concatenate_datasets([positive, negative_oversampled])
    return balanced_dataset

balanced_dataset = oversample_dataset(dataset)
train_test_split = balanced_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

Filter:   0%|          | 0/500000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [ ]:
test_dataset = test_dataset.select(range(35000))

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", num_labels=2)
tokenizer_sentiment = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Some weights of MBartForSequenceClassification were not initialized from the model checkpoint at facebook/mbart-large-50-many-to-many-mmt and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer_sentiment(examples['text'], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)
tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/777015 [00:00<?, ? examples/s]

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    print("Тип pred.predictions:", type(pred.predictions))
    print("Содержимое pred.predictions:", pred.predictions)
    if isinstance(pred.predictions, tuple):
        logits = pred.predictions[0]
    else:
        logits = pred.predictions
    print("Тип logits:", type(logits))
    if isinstance(logits, np.ndarray):
        logits = torch.tensor(logits)
    elif not isinstance(logits, torch.Tensor):
        raise ValueError(f"Неожиданный тип logits: {type(logits)}")
    preds = torch.nn.functional.softmax(logits, dim=-1).argmax(dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
model.to(torch.device("cuda"))
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="/content/drive/MyDrive/Coursework/Bart_fine-tuning/logs_sentiment",
    logging_steps=500,
    learning_rate=5e-5,
    fp16=True,
    metric_for_best_model="eval_loss",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

In [ ]:
torch.cuda.empty_cache()
trainer.train()
model.save_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment")
tokenizer_sentiment.save_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment")
tokenizer_sentiment = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment")

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment",
    per_device_eval_batch_size=1,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

torch.cuda.empty_cache()

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
gc.collect()
torch.cuda.empty_cache()
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment",
    per_device_eval_batch_size=1,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

In [ ]:
print("\033[1mИтоговые метрики на тестовом наборе:", trainer.evaluate())

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Тип pred.predictions: <class 'tuple'>
Содержимое pred.predictions: (array([[ 4.1289062, -3.7460938],
       [-3.6347656,  2.9296875],
       [-3.3378906,  2.8417969],
       ...,
       [ 2.4355469, -1.9912109],
       [ 2.4375   , -2.0097656],
       [ 3.234375 , -2.8554688]], dtype=float32), array([[[ 0.03288536,  0.00673956,  0.03273522, ..., -0.01258551,
          0.00289448,  0.0312844 ],
        [ 0.44654122, -0.40922374, -0.4928495 , ..., -0.12400982,
          0.64697295, -0.7163965 ],
        [ 0.58472836, -0.57851356, -0.19972192, ...,  0.25937948,
          0.8724377 , -0.7202352 ],
        ...,
        [ 0.23059681, -0.14705077, -0.5379977 , ...,  0.02895649,
          0.30127743, -0.6344463 ],
        [ 0.3008063 , -0.26684624, -0.42920187, ..., -0.13090932,
          0.18152262, -0.73752904],
        [ 0.27605772, -0.24497055, -0.53759897, ..., -0.3012034 ,
          0.14905895, -0.7389246 ]],

       [[ 0.04631447,  0.02359814, -0.01049729, ..., -0.05363747,
         -0.

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: d-d-pushkarev-d-d (d-d-pushkarev-d-d-hse-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Итоговые метрики на тестовом наборе: {'eval_loss': 0.1402750015258789, 'eval_model_preparation_time': 0.0064, 'eval_accuracy': 0.9641428571428572, 'eval_f1': 0.9640761414054673, 'eval_precision': 0.9698784772216783, 'eval_recall': 0.958342818119736, 'eval_runtime': 5760.5394, 'eval_samples_per_second': 6.076, 'eval_steps_per_second': 6.076}


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", num_labels=2)
tokenizer_sentiment = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment",
    per_device_eval_batch_size=1,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
gc.collect()
torch.cuda.empty_cache()
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment",
    per_device_eval_batch_size=1,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

In [ ]:
print("\033[1mИтоговые метрики на тестовом наборе:", trainer.evaluate())

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Тип pred.predictions: <class 'tuple'>
Содержимое pred.predictions: (array([[ 0.0111618 ,  0.20715332],
       [ 0.45385742,  0.50634766],
       [ 0.1595459 ,  0.21252441],
       ...,
       [-0.18078613, -0.0814209 ],
       [-0.46411133,  0.49975586],
       [-0.5834961 , -0.01107788]], dtype=float32), array([[[ 2.10841578e-02,  1.42508023e-03, -3.20415339e-03, ...,
         -2.83735897e-02, -5.63972583e-03, -8.13428592e-03],
        [-1.27015567e+00,  5.57727993e-01, -4.31161433e-01, ...,
          9.66910720e-02,  4.80649799e-01,  1.08139992e+00],
        [-1.13831736e-01, -2.79016286e-01, -2.73533434e-01, ...,
          9.01037991e-01,  9.05138195e-01,  1.86407462e-01],
        ...,
        [ 3.67573440e-01,  1.09572875e+00, -9.13495421e-01, ...,
          1.00908089e+00,  1.16693509e+00, -1.89244617e-02],
        [ 1.09050822e+00,  5.98946929e-01, -1.18075006e-01, ...,
         -4.31496471e-01,  5.81634045e-01, -4.84199136e-01],
        [ 1.18120080e-02, -5.25466027e-03,  1.6017

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: d-d-pushkarev-d-d (d-d-pushkarev-d-d-hse-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Итоговые метрики на тестовом наборе: {'eval_loss': 0.7163339257240295, 'eval_model_preparation_time': 0.0063, 'eval_accuracy': 0.4439142857142857, 'eval_f1': 0.48055726066881954, 'eval_precision': 0.45248027340805147, 'eval_recall': 0.5123491918961985, 'eval_runtime': 5782.9183, 'eval_samples_per_second': 6.052, 'eval_steps_per_second': 6.052}


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment")
tokenizer_sentiment = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment")

In [ ]:
model.to(torch.device("cuda"))
model.eval()
cnt = 0
need = 5
print("\033[1m\033[33mПример работы модели на первых 5 примерах из тестовой выборки:\033[0m")
for review in  test_dataset:
    inputs = tokenizer_sentiment(review['text'], padding="max_length", truncation=True, max_length=128, return_tensors="pt").to(torch.device("cuda"))
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=-1).item()
    sentiment = "Положительный" if prediction == 1 else "Отрицательный"
    wrapped_text = textwrap.fill(review['text'], width=140, break_long_words=False)
    print(f"Отзыв: {wrapped_text}")
    print(f"Предсказанная тональность: {sentiment}")
    print(f"Истинная тональность: {'Положительный' if review['label'] == 1 else 'Отрицательный'}\n")
    print()
    if cnt == need:
      break
    cnt += 1

Пример работы модели на первых 5 примерах из тестовой выборки:
Отзыв: Все было чудесно на этапе согласования, на этом чудеса закончились! Мы заказали 3 двери Инвизибл под покраску. Оплатили заранее, далее
заказали доставку и установка нам необходима была в 2 этапа, так как такие двери ставятся в момент когда полы сделаны, а стены еще нет. Мы
попросили сначала поставить 1 дверь в ванную и далее оставшиеся 2, на что получили ответ, что установка 1 двери для мастера неинтересный
выезд и такой выезд будет стоить на 2000 руб. дороже, неприятно, но ладно. Дверь поставили. Далее ее сняли, чтобы проделать все необходимые
отделочные работы. После чего был повторный выезд на установку 2-х оставшихся дверей, мастер повесил и третью, что была установлена ранее по
повышенном тарифу и за то, чтобы повесить дверь он взял еще 2000 руб, и тут же выясняется что коробка стоит криво, и никак это не
отрегулировать, нужно сносить все и ставить заново!!! И здесь пошло молчание от офиса кто будет все переделы

In [ ]:
dataset = load_dataset("RussianNLP/Mixed-Summarization-Dataset")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

README.md:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/538M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/605k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/197561 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/258 [00:00<?, ? examples/s]

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned")
tokenizer = MBart50Tokenizer.from_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned")

In [ ]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50Tokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "ru_RU"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["text"],
        max_length=max_input_length,
        truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"],
            max_length=max_target_length,
            truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Map:   0%|          | 0/197561 [00:00<?, ? examples/s]

Map:   0%|          | 0/258 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=3e-5,
    fp16=True,
    logging_steps=1000,
    save_steps=2000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)


In [ ]:
trainer.train()
trainer.save_model("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize")
tokenizer.save_pretrained("/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: d-d-pushkarev-d-d (d-d-pushkarev-d-d-hse-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1000,2.683400
2000,2.560200
3000,2.519000
4000,2.492700
5000,2.451600
6000,2.410400
7000,2.415000


Step,Training Loss
1000,2.683400
2000,2.560200
3000,2.519000
4000,2.492700
5000,2.451600
6000,2.410400
7000,2.415000
8000,2.378600
9000,2.373300
10000,2.335800


('/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize/tokenizer_config.json',
 '/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize/special_tokens_map.json',
 '/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize/sentencepiece.bpe.model',
 '/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize/added_tokens.json')

In [ ]:
model_path = "/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize"
model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = MBart50Tokenizer.from_pretrained(model_path)

tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "ru_RU"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"\033[1m\033[33mПример работы модели на задаче суммаризации:\033[0m")
texts = ["Все было чудесно на этапе согласования, на этом чудеса закончились! Мы заказали 3 двери Инвизибл под покраску. Оплатили заранее, далее заказали доставку и установка нам необходима была в 2 этапа, так как такие двери ставятся в момент когда полы сделаны, а стены еще нет. Мы попросили сначала поставить 1 дверь в ванную и далее оставшиеся 2, на что получили ответ, что установка 1 двери для мастера неинтересный выезд и такой выезд будет стоить на 2000 руб. дороже, неприятно, но ладно. Дверь поставили. Далее ее сняли, чтобы проделать все необходимые отделочные работы. После чего был повторный выезд на установку 2-х оставшихся дверей, мастер повесил и третью, что была установлена ранее по повышенном тарифу и за то, чтобы повесить дверь он взял еще 2000 руб, и тут же выясняется что коробка стоит криво, и никак это не отрегулировать, нужно сносить все и ставить заново!!! И здесь пошло молчание от офиса кто будет все переделывать? А главное за что я заплатила 22,5 руб. за установку чего, кривизны? Офис никаких комментариев не давал, поставили криво и забыли, крайне неприятная ситуация. Ответили только после того, как я написала отрицательный отзыв. Это не обычные двери, а те, что монтируются в стену здесь доборами и планками ничего не исправить, нужно ломать и строить заново! Ванная - там вообще керамогранит уже на стенах!Безответственные сотрудники! Заказывали в салоне Коммунарка/Бутово, крайне не рекомендую связываться с данной компанией!\nДополню свой и без того отрицательный отзыв, еще одним минусом сотрудничества с данной компанией. Устанавливая ручки и замки на двери, мы поняли, что на обратной двери (она больше стандартных) неподходящей длины четырехгранник, на что в офисе мне ответили, что его можно купить в магазине и в комплектацию он не входит, хорошо пену покупать не нужно и петли дверные. И более того, побродив по просторам интернета, мы поняли, что этих четырехгранников нигде нет, о чем я снова написала сотрудникам магазина, после чего получила ответ, что у них их не было и вот они появились, могут нам продать! Продать, то что входит в комплект! После того, как я подготовила досудебную претензию, магазин все же решил довезти недостающий четырехгранник. Вывод, чего-то добиться можно, но со скандалами и тратой нервов..", "Каждый раз, посещая этот магазин, чувствую неприятную атмосферу, из-за нервных кассиров, был свидетелем, когда кассир нагрубила пожилой женщине, потребовав у нее быстрее расплатиться так-как она задерживает очередь, она терпеливо с дрожащими руками расплатилась, после этого осадок остался не приятный. Интересно Чижик набирает кассиров именно с качествами нервно больных людей? Ужасный персонал!\n"]
for text in texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    forced_bos_id = tokenizer.lang_code_to_id["ru_RU"]
    summary_ids = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        forced_bos_token_id=forced_bos_id,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    wrapped_text = textwrap.fill(text, width=140, break_long_words=False)
    wrapped_summary = textwrap.fill(summary, width=140, break_long_words=False)

    print(f"Длина отзыва: {len(text)}; Отзыв: {wrapped_text}")
    print(f"Длинна суммаризации: {len(summary)}; Суммаризация отзыва:{wrapped_summary}")
    print()

Пример работы модели на задаче суммаризации:
Длина отзыва: 2260; Отзыв: Все было чудесно на этапе согласования, на этом чудеса закончились! Мы заказали 3 двери Инвизибл под покраску. Оплатили заранее, далее
заказали доставку и установка нам необходима была в 2 этапа, так как такие двери ставятся в момент когда полы сделаны, а стены еще нет. Мы
попросили сначала поставить 1 дверь в ванную и далее оставшиеся 2, на что получили ответ, что установка 1 двери для мастера неинтересный
выезд и такой выезд будет стоить на 2000 руб. дороже, неприятно, но ладно. Дверь поставили. Далее ее сняли, чтобы проделать все необходимые
отделочные работы. После чего был повторный выезд на установку 2-х оставшихся дверей, мастер повесил и третью, что была установлена ранее по
повышенном тарифу и за то, чтобы повесить дверь он взял еще 2000 руб, и тут же выясняется что коробка стоит криво, и никак это не
отрегулировать, нужно сносить все и ставить заново!!! И здесь пошло молчание от офиса кто будет все переде

In [ ]:
class ReviewSummarizer(nn.Module):
  def __init__(
      self,
      generative_model_path="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize",
      classification_model_path="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment",
      gen_tokenizer_path="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_summarize",
      class_tokenizer_path="/content/drive/MyDrive/Coursework/Bart_fine-tuning/mbart_finetuned_sentiment"):
      super(ReviewSummarizer, self).__init__()
      self.gen_model = MBartForConditionalGeneration.from_pretrained(generative_model_path)
      self.class_model = AutoModelForSequenceClassification.from_pretrained(classification_model_path, num_labels=2)
      self.gen_tokenizer = AutoTokenizer.from_pretrained(gen_tokenizer_path)
      self.class_tokenizer = AutoTokenizer.from_pretrained(class_tokenizer_path)
      self.num_labels = 2

      self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      self.gen_model.to(self.device)
      self.class_model.to(self.device)

  def summarize(self, text, max_length=128, summary_max_length=50):
      class_inputs = self.class_tokenizer(
          text,
          padding="max_length",
          truncation=True,
          max_length=max_length,
          return_tensors="pt"
      ).to(self.device)

      with torch.no_grad():
          prediction = torch.argmax(self.class_model(**class_inputs).logits, dim=-1).item()
          sentiment = "положительный" if prediction == 1 else "отрицательный"

      gen_inputs = self.gen_tokenizer(
          text,
          padding="max_length",
          truncation=True,
          max_length=max_length,
          return_tensors="pt"
      ).to(self.device)

      with torch.no_grad():
          gen_outputs = self.gen_model.generate(
              **gen_inputs,
              max_length=128,
              num_beams=7,
              early_stopping=True
          )
          summary = self.gen_tokenizer.decode(gen_outputs[0], skip_special_tokens=True)
          wrapped_summary = textwrap.fill(summary, width=120, break_long_words=False)
      print(f"Данный отзыв: {sentiment}")
      print(f"Суммаризация отзыва: {wrapped_summary}")

In [ ]:
summarizer = ReviewSummarizer()

In [ ]:
text = 'В этой кофейне уютная атмосфера. Чисто. Вежливый персонал. Варят вкусный, разнообразный кофе. Интерьер в стиле "минимализм" и это отлично: от прекрасного кофе ничто не должно отвлекать)'

summarizer.summarize(text)

Данный отзыв: положительный
Суммаризация отзыва: В кофейне уютная атмосфера, вежливый персонал и разнообразный кофе. Интерьер в стиле "минимализм".
